# Assignment 2 - Vehicle Routing Problem
JM0100-M-6 Business Analytics  
Myrthe Wouters  
u1273195

In [1]:
# Global imports
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import math
import operator
from copy import deepcopy
import random
import copy
from deap import base, creator, tools, algorithms

In [2]:
# Load data
STORES = pd.read_excel('Data Excercise 2 - EMTE stores - BA 2020-1.xlsx', index_col='City Nr.')

## Exercise 2.1

In [3]:
class Location:
    
    # Set coordinates of headquarters as class variable
    coords_hq = STORES.loc[0, 'Lat'], STORES.loc[0, 'Long']
    
    def __init__(self, nr, name, lat, long, store_type):
        self.nr = nr
        self.name = name
        self.lat = lat
        self.long = long
        self.store_type = store_type
        self.visited = False
        
    @property
    def coords(self):
        """Defines coordinates of a location"""
        return (self.lat, self.long)
    
    @property
    def distance_hq(self):
        """Defines distance from location to headquarters"""
        dist = round(haversine(self.coords_hq, self.coords))
        return dist
    
    @property
    def is_jumbo(self):
        """Boolean value that shows if location is a Jumbo location"""
        return True if self.store_type=='Jumbo' else False

In [4]:
def get_params(df, index):
    """
    Function to get all necessary variables from stores dataframe to instantiate a Location instance for every store
    """
    params = df.iloc[index][['Name', 'Lat', 'Long', 'Type']].values
    name, lat, long, store_type = params
    return index, name, lat, long, store_type

In [5]:
class Route:
   
    # Set class variables
    max_working_mins = 11*60
    max_opening_mins = 8*60
    speed_kmh = 90
    
    def __init__(self, farthest_store, hq):
        self.inner_route=[]
        self.farthest_store = farthest_store
        self.hq = hq
    
    @property
    def meeting_time(self):
        """Defines the meeting time at individiual stores in the route"""
        return 30 if self.farthest_store.is_jumbo else 20
    
    @property
    def store_types(self):
        """Defines the type of stores in this route"""
        return 'Jumbo' if self.farthest_store.is_jumbo else 'Coop or other'
    
    @property
    def full_route(self):
        """Defines full route of the route, including the start and finish at headquarters"""
        full_route = self.inner_route.copy()
        full_route.insert(0, self.hq)
        full_route.append(self.hq)
        return full_route
    
    @property
    def total_distance(self):
        """Defines total distance in km of the route"""
        return sum(Route.distances(self.full_route))
    
    @property
    def cumsum(self):
        """Defines cumulative sum of distances at each location"""
        cumsum = list(np.cumsum(Route.distances(self.full_route)))
        cumsum.insert(0, 0)
        return cumsum
    
    @property
    def total_visit_time(self):
        """Defines total time spend in all stores of route together"""
        return len(self.inner_route) * self.meeting_time
    
    @property
    def working_hours_constraint(self):
        """Defines if route meets the constraint of John's working hours"""
        return (Route.travel_time(self.full_route) + self.total_visit_time) <= Route.max_working_mins
    
    @property
    def opening_hours_constraint(self):
        """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
        return (Route.travel_time(self.inner_route) + self.total_visit_time) <= Route.max_opening_mins
    
    @property
    def all_hours_constraints(self):
        """Defines if route meets both constraints John's working hours and visiting hours at every store"""
        return self.working_hours_constraint and self.opening_hours_constraint
    
    @staticmethod
    def calc_dist(loc1, loc2):
        """Defines rounded distance (km) between two locations"""
        dist = round(haversine(loc1.coords, loc2.coords))
        return dist
    
    @staticmethod
    def distances(route):
        """Defines all distances between consecutive locations in a route"""
        distances = [Route.calc_dist(route[idx], route[idx+1]) for 
                          idx, _ in enumerate(route[:-1])]
        return distances
    
    @staticmethod
    def dist_to_min(km):
        """Defines the duration of a route in minutes"""
        speed_kmm = (Route.speed_kmh)/60
        minutes = round(km/speed_kmm)
        return minutes
    
    @staticmethod
    def travel_time(route):
        """
        Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route
        """
        minutes_between_locs = [Route.dist_to_min(dist) for dist in Route.distances(route)]
        travel_time = sum(minutes_between_locs)
        return travel_time
    
    def insert(self, pos, item):
        """Insert location at given position in route"""
        self.inner_route.insert(pos, item)
    
    def remove(self, item):
        """Remove location from route"""
        self.inner_route.remove(item)

In [6]:
class RoutePlanner:
    
    def __init__(self, locations):
        self.locations = locations[1:]
        self.hq = locations[0]
        
    @property
    def loc_distances_hq(self):
        """Defines the list of stores sorted in decreasing order based on distance from headquarter"""
        sorted_dist = sorted(self.locations, key=lambda x: x.distance_hq, reverse=True)
        return sorted_dist
    
    @property
    def to_visit(self):
        """Defines the list of stores that still need to be visited in a future route"""
        stores_to_visit = [loc for loc in self.locations if loc.visited==False]
        return stores_to_visit
    
    @property
    def current_farthest_store(self):
        farthest_store = [loc for loc in self.loc_distances_hq if loc.visited==False][0]
        return farthest_store
    
    @property
    def stores_left(self):
        """Defines if there are any stores left to visit"""
        return len([loc for loc in self.locations if loc.visited==False and loc!= self.hq])>0
    
    @staticmethod
    def calc_dist(loc1, loc2):
        """Defines rounded distance (km) between two locations"""
        dist = round(haversine(loc1.coords, loc2.coords))
        return dist
    
    def potential_stores(self, farthest_store):
        """Defines potential stores for a given route with given farthest location"""
        if farthest_store.is_jumbo:
            potential_stores = [loc for loc in self.to_visit if loc.store_type=='Jumbo' and loc!=farthest_store]
        else: 
            potential_stores = [loc for loc in self.to_visit if loc.store_type!= 'Jumbo' and loc!=farthest_store]
        return potential_stores

    def dist_farthest_store(self, farthest_store):
        """Defines distances for all potential stores from farthest store in a route in sorted order ascending"""
        distances = [(RoutePlanner.calc_dist(farthest_store, potential_store), potential_store) 
                     for potential_store in self.potential_stores(farthest_store)]
        distances.sort(key=operator.itemgetter(0))
        sorted_dist = [potential_store for dist, potential_store in distances]
        return sorted_dist
    
    def plan_route(self, route):
        """Plans individual routes"""
        
        # Insert current farthest store to be visited in rote
        route.insert(0, route.farthest_store)
        
        # All potential locations in ascending order
        for i in self.dist_farthest_store(route.farthest_store):
            best_dist = math.inf
            best_pos = None
            
            # Try location on every position in route
            for pos in range(len(route.inner_route)+1):
                
                # Insert location at position
                route.insert(pos, i)
                
                # Check if location at given position meets constraints and;
                # Check if this position is better than all previous viable positions for this location
                if route.all_hours_constraints and route.total_distance<best_dist:
                    best_dist = route.total_distance
                    best_pos = pos
                
                # Remove location from route
                route.remove(i)
            
            # If there is a viable position, insert location at best viable position in route
            if best_pos!=None:
                route.insert(best_pos, i)
        
        # Set all locations that are inserted to route to visited
        for loc in route.inner_route:
            loc.visited = True
            
    def solve(self):
        """Solves VRP for all locations"""
        
        # Set route number
        j = 0
        routes = {} # Empty dict to store routes
        while self.stores_left:
            j += 1 # Update route number
            farthest_store = self.current_farthest_store # Assign current farthest store
            route = Route(farthest_store, self.hq) # Instantiate route instance with current farthest store
            self.plan_route(route) # Plan this route
            routes[j] = route # Store route with route as key in dict
        return routes

In [7]:
def to_Excel(planned_routes, exercise_nr, save=True, file_name=None):
    """Creates dataframe with needed information from solved VRP, saves to Excel file if needed"""
    
    # Dictionary to store data from all routes
    data = {'Route Nr.': [], 
            'City Nr.': [],
            'City Name': [], 
            'Total Distance in Route (km)': [],
            'Total Distance (km)': []}
    
    # Define total kilometers traveled at start of the route in previous routes
    total_km_at_start = 0
    
    # All routes in planned routes
    for nr, route in planned_routes.items():
        cumsum_loc = 0  # Define current cumulative sum location
        
        # Add data for location to dictionary
        # Because routes are instances of Route class in Exercise 1, but of RouteDLL class in Exercise 2,
        # We need to make an if-statement here
        if exercise_nr == 1:
            for location in route.full_route:
            
                data['Route Nr.'].append(nr)
                data['City Nr.'].append(location.nr)
                data['City Name'].append(location.name)
                data['Total Distance in Route (km)'].append(route.cumsum[cumsum_loc])
                data['Total Distance (km)'].append(route.cumsum[cumsum_loc] + total_km_at_start)
        
        # For Exercise 2, routes are instances of RouteDLL class (doubly linked lists)
        if exercise_nr == 2:
            temp = route.start # Initialise temp
            
            while (temp):
                data['Route Nr.'].append(nr)
                data['City Nr.'].append(temp.data.nr)
                data['City Name'].append(temp.data.name)
                data['Total Distance in Route (km)'].append(route.cumsum[cumsum_loc])
                data['Total Distance (km)'].append(route.cumsum[cumsum_loc] + total_km_at_start)
                
                temp = temp.next
            
                cumsum_loc += 1 # Update current cumulative sum location
        
        # Update total kilometers traveled at start of the route in previous routes
        total_km_at_start += route.cumsum[-1] 
    
    df = pd.DataFrame.from_dict(data) # Save data to DataFrame
    
    # If needed, save data to Excel file
    if save:
        df.to_excel(file_name, index=False)
            
    return df

In [8]:
# Instantiate location instance for every store
locations = [Location(*get_params(STORES, index)) for index, _ in STORES.iterrows()]

# Instantiate planner object
planner = RoutePlanner(locations)
planned_routes_1 = planner.solve() # Solve VRP

# Save results to Excel file
df_q21 = to_Excel(planned_routes_1, 1, file_name='../excel-files/Ex2.1-1273195.xls')

## Exercise 2.2

In [9]:
locations = [Location(*get_params(STORES, index)) for index, _ in STORES.iterrows()]

In [10]:
class Node:
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

In [11]:
HQ = locations[0]

In [12]:
class RouteDLL:
    hq_start = Node(HQ)
    hq_end = Node(HQ)
    
    max_working_mins = 11*60
    max_opening_mins = 8*60
    speed_kmh = 90
    
    def __init__(self, farthest_store):
        self.start = RouteDLL.hq_start
        self.end = RouteDLL.hq_end
        self.start.next = self.end
        self.end.prev = self.start
        self.farthest_store = farthest_store
        self.is_jumbo = self.farthest_store.is_jumbo
        
    @property
    def meeting_time(self):
        """Defines the meeting time at individiual stores in the route"""
        return 30 if self.farthest_store.is_jumbo else 20
    
    @property
    def length(self):
        """Defines number of stores in the route"""
 
        temp = self.start # Initialise temp 
        count = 0 # Initialise count 
  
        # Loop while end of linked list is not reached 
        while (temp): 
            count += 1
            temp = temp.next
            
        return count 
    
    @property
    def full_route(self):
        """Returns the order of stores in the full route as a list"""
        
        temp = self.start
        route = []
        
        while (temp):
            route += [temp.data.nr]
            temp = temp.next
            
        return route
    
    @property
    def total_visit_time(self):
        """Defines total time spend in all stores of route together"""
        return ((self.length)-2) * self.meeting_time
    
    @property
    def store_type_constraint(self):
        """Defines if all stores in route are of same type"""
        temp = self.start.next
        
        while temp != self.end:
            if temp.data.is_jumbo != self.is_jumbo:
                return False
            temp=temp.next
        
        return True
    
    @property
    def working_hours_constraint(self):
        """Defines if route meets the constraint of John's working hours"""
        return (self.total_distance_time()['total_travel_time'] + self.total_visit_time) <= RouteDLL.max_working_mins
    
    @property
    def opening_hours_constraint(self):
        """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
        return (self.inner_distance_time()['total_travel_time'] + self.total_visit_time) <= RouteDLL.max_opening_mins
    
    @property
    def all_hours_constraints(self):
        """Defines if route meets both constraints John's working hours and visiting hours at every store"""
        return self.working_hours_constraint and self.opening_hours_constraint
    
    @property
    def is_valid(self):
        """
        Defines if route meets all constraints, that is:
        * Constraint of John's working hours (11 hours max)
        * Constraint of visiting hours at every store (9:00-17:00, that is, 8 hours max)
        * Constraint of every store in route being either of type Jumbo or of type other (Coop or other)
        """
        return self.all_hours_constraints and self.store_type_constraint
    
    @property
    def total_distance(self):
        """Defines total distance of total route"""
        return self.total_distance_time()['total_distance']
    
    @property
    def cumsum(self):
        """Defines cumulative sum of distances at each location"""
        temp = self.start # Initialise temp
        distance = 0
        cumsum = []
        
        while temp.next:
            # Calculate distance between current and next node
            dist = RouteDLL.calc_dist(temp.data, temp.next.data)
            
            # Add dist to distance
            distance += dist
            #Append cumulative distance until this node to cumsum list
            cumsum.append(distance)
            
            # Move to next node
            temp = temp.next
        
        # Insert cumsum of 0 at start at HQ
        cumsum.insert(0, 0)
        return cumsum
    
    @staticmethod
    def calc_dist(loc1, loc2):
        """Defines rounded distance (km) between two locations"""
        dist = round(haversine(loc1.coords, loc2.coords))
        return dist
    
    @staticmethod
    def dist_to_min(km):
        """Defines the duration of a route in minutes"""
        speed_kmm = (RouteDLL.speed_kmh)/60
        minutes = round(km/speed_kmm)
        return minutes
    
    def total_distance_time(self):
        """Defines total distance and travel time of total route"""
        temp = self.start # Initialise temp
        distance = 0
        travel_time = 0
        
        while temp.next:
            # Calculate distance and time between current and next node
            dist = RouteDLL.calc_dist(temp.data, temp.next.data)
            time = RouteDLL.dist_to_min(dist)
            
            # Add dist and time to distance and travel_time
            distance += dist
            travel_time += time
            
            # Move to next node
            temp = temp.next
            
        return {'total_distance': distance,
                'total_travel_time': travel_time}
    
    def inner_distance_time(self):
        """Defines total distance and travel time of inner route (i.e., route without start and end at hq)"""
        temp = self.start.next # Initialise temp
        distance = 0
        travel_time = 0
        
        while temp.next != self.end:
            # Calculate distance and time between current and next node
            dist = RouteDLL.calc_dist(temp.data, temp.next.data)
            time = RouteDLL.dist_to_min(dist)
            
            # Add dist and time to distance and travel_time
            distance += dist
            travel_time += time
            
            # Move to next node
            temp = temp.next
        
        return {'total_distance': distance,
                'total_travel_time': travel_time}    
        
    def insert_after(self, prev_node, new_node):
        """Insert node at given position in route"""
        
        # Check if prev_node exists
        if prev_node is None:
            print("This node doesn't exist in DLL")
            return
        
        # Make next of new node as next of prev_node
        new_node.next = prev_node.next
        
        # Make the next node of prev_node as new_node
        prev_node.next = new_node
        
        # Make the prev_node as previous of new_node
        new_node.prev = prev_node
        
        # Change previous of new_node's next node
        if new_node.next is not None:
            new_node.next.prev = new_node
            
    def remove(self, dele):
        """Remove node from route"""
        
        # Change previous pointer of dele's next node:
        dele.next.prev = dele.prev 
        
        # Change next pointer of dele's previous node:
        dele.prev.next = dele.next
        
        # Set dele's previous and next pointers to None
        dele.next=None
        dele.prev=None

### Set Route instances of Ex 2.1 to RouteDLL instances of Ex 2.2

In [13]:
planned_routes_dll = []

for route in planned_routes_1.values():
    route_dll = RouteDLL(route.farthest_store)
    
    curr = route_dll.start # Initialise for insertion
    
    # Insert each location in inner route of route to route_dll
    for loc in route.inner_route:
        node_loc = Node(loc)
        route_dll.insert_after(curr, node_loc)
        curr = node_loc
    
    # Append a deep copy of the RouteDLL instance to a list of planned routes of type RouteDLL
    planned_routes_dll.append(deepcopy(route_dll))

In [14]:
class Solver:
    
    def __init__(self, planned_routes):
        self.planned_routes = planned_routes
        
    @property
    def all_routes_valid(self):
        """Checks if all planned routes are valid, i.e., meets all constraints"""
        for route in self.planned_routes:
            if not route.is_valid:
                return False
        return True
    
    @property
    def total_distance(self):
        """Defines total distance of all planned routes together"""
        dist = 0
        for route in self.planned_routes:
            dist += route.total_distance
        return dist
        
    @staticmethod
    def swap_nodes(node_1, node_2):
        """Swaps node 1 with node 2, i.e., swap data of node 1 with data of node 2"""
        # Nothing to do if x and y are same 
        if node_1 == node_2: 
            return 
    
        temp = deepcopy(node_1.data)
        
        node_1.data = node_2.data
        node_2.data = temp
        
    @staticmethod
    def move_node_after(node, node_to_move_after):
        """Move position of node to position after node_to_move_after"""
        
        # If node is is equal to node_to_move_after, do nothing
        if node == node_to_move_after:
            return
        
        # Remove node at original position
        ## Change node's previous node next pointer to node's next node
        node.prev.next = node.next
        
        ## Change node's next node previous pointer to node's previous node
        node.next.prev = node.prev
        
        # Insert node at new position
        ## Make next of new node as next of prev_node
        node.next = node_to_move_after.next
        
        ## Make the next node of prev_node as new_node
        node_to_move_after.next = node
        
        ## Make the prev_node as previous of new_node
        node.prev = node_to_move_after
        
        ## Change previous of new_node's next node
        node.next.prev = node
        
    @staticmethod
    def undo_move(route_org, route_curr, node, orig_before_node):
        """Undo the move of node"""
        
        route_curr.remove(node)
        route_org.insert_after(orig_before_node, node)
    
    def all_swaps(self, changed_routes):
        """Computes all possible swaps for all nodes in planned routes"""
        
        # Dictionary to store the swaps that lead to new improvements in this iteration
        improvements = {}
        
        curr_dist = deepcopy(self.total_distance) # store the distance of all planned routes now together 
        
        # Check for each location
        for idx_route, route in enumerate(self.planned_routes):
            # Set node to compute all swaps with
            curr_loc = route.start.next
            
            while curr_loc != route.end:
                # Compute only swaps with nodes in current route and routes after, in order to prevent doing double
                # the work
                for check_route in self.planned_routes[idx_route:]:
                    
                    # If none of the routes the two nodes are in changed compared to the last iteration,
                    # we have saved the improvement of the swap and thus do not have to compute it again
                    if (route not in changed_routes) and (check_route not in changed_routes):
                        continue
                    
                    # If any of the routes the two nodes are in changed, we have to recompute the improvement of the
                    # swap
                    else: 
                        if route == check_route:
                            check_loc = curr_loc.next # Only check for nodes after current node
                        else:
                            check_loc = check_route.start.next
                        
                        while check_loc != check_route.end:
                            Solver.swap_nodes(curr_loc, check_loc) # swap two nodes
                        
                            improv = curr_dist - (self.total_distance) # check if there is an improvement
                            
                            # If the swap leads to an improvement and results in all valid routes 
                            # save improvement and details in improvements dictionary
                            if (improv > 0) and self.all_routes_valid:
                                improvements[(curr_loc, check_loc)] = {}
                                improvements[(curr_loc, check_loc)]['improv'] = improv
                                improvements[(curr_loc, check_loc)]['routes'] = (route, check_route)
                            
                            # Swap nodes back
                            Solver.swap_nodes(check_loc, curr_loc)
                            
                            # Move to next location to swap with
                            check_loc = check_loc.next
                            
                # Check swaps for next location
                curr_loc = curr_loc.next
        return improvements
    
    def all_moves(self, changed_routes):
        """Computes all possible swaps for all nodes in planned routes"""
        
        # Dictionary to store the moves that lead to new improvements in this iteration
        improvements = {}
        
        curr_dist = deepcopy(self.total_distance) # store the distance of all planned routes now together 
        
        # Check for each location
        for route in self.planned_routes:
            # Set node to compute all moves for
            curr_loc = route.start.next
            
            while curr_loc != route.end:
                orig_before_node = curr_loc.prev
                
                for check_route in self.planned_routes:
                    # Set node we will move curr_loc after
                    move_after = check_route.start
                    
                    # If none of the routes the two nodes are in changed compared to the last iteration,
                    # we have saved the improvement of the move and thus do not have to compute it again
                    if (route not in changed_routes) and (check_route not in changed_routes):
                        continue
                    
                    else:
                        while move_after != check_route.end:
                        
                            Solver.move_node_after(curr_loc, move_after) # move curr_loc after move_after

                            improv = curr_dist - (self.total_distance) # check if there is an improvement
                            
                            # If the move leads to an improvement and results in all valid routes,
                            # save improvement and details in improvements dictionary
                            if (improv > 0) and self.all_routes_valid:
                                improvements[(curr_loc, move_after)] = {}
                                improvements[(curr_loc, move_after)]['improv'] = improv
                                improvements[(curr_loc, move_after)]['routes'] = (route, check_route)
                            
                            # Undo the move
                            solver.undo_move(route, check_route, curr_loc, orig_before_node)
                            
                            # Move to next location to move after
                            move_after = move_after.next
                
                # Check moves for next location
                curr_loc = curr_loc.next

        return improvements
    
    def optimize(self):
        
        operations = 0 # Initialise nr of operation to lead to optimal result
        swap_improv = {} # Dictionary to save all swap improvements
        move_improv = {} # Dictionary to save all move improvements
        
        changed_routes = self.planned_routes # Initialise routes that changed since last iteration
        
        
        while True:
            # Compute all move and swaps for this iteration
            iter_swap_improv = self.all_swaps(changed_routes)
            iter_move_improv = self.all_moves(changed_routes)
            
            # Update dictionaries with swap and move improvements with improvements of changed routes 
            # during last iteration
            swap_improv.update(iter_swap_improv)
            move_improv.update(iter_move_improv)
            
            # Sort swap improvements on improvement value decreasing
            swap_improv_list = [(values['improv'], swap, values['routes']) for swap, values in swap_improv.items()]
            swap_improv_sorted = sorted(swap_improv_list, key=operator.itemgetter(0), reverse = True)
            
            # If there is a swap that leads to an improvement, set best_swap_improv to improvement of best swap
            if len(swap_improv_sorted) > 0:
                best_swap_improv = swap_improv_sorted[0][0]
            else:
                best_swap_improv = 0 # Else set to 0, as there is no swap improvement
            
            # Sort move improvements on improvement value decreasing
            move_improv_list = [(values['improv'], move, values['routes']) for move, values in move_improv.items()]
            move_improv_sorted = sorted(move_improv_list, key=operator.itemgetter(0), reverse = True)
            
            # If there is a move that leads to an improvement, set best_move_improv to improvement of best move
            if len(move_improv_sorted) > 0:
                best_move_improv = move_improv_sorted[0][0]
            else:
                best_move_improv = 0
            
            # If there is no swap or move improvement anymore, we have found the optimal solution for now
            if (len(swap_improv) + len(move_improv))==0:
                break
            
            # If the best improvement is a move improvement, compute the best move
            if best_move_improv > best_swap_improv:
                self.move_node_after(*move_improv_sorted[0][1])
                operations+=1 # Increase number of operations by 1
                print('Moved {} after {}'.format(move_improv_sorted[0][1][0].data.name,
                                                 move_improv_sorted[0][1][1].data.name))
                
                # Keep track of the routes that have changed during the move operation
                changed_routes = list(move_improv_sorted[0][2])
                
                # Delete moves in changed_routes from move_improv dictionary, as they have to be computed again in 
                # next iteration
                for move in move_improv_sorted:
                    if any(True for route in changed_routes if route in move[2]):
                        del move_improv[move[1]]
                
                # Delete swaps in changed_routes from swap_improv dictionary, as they have to be computed again in 
                # next iteration
                for swap in swap_improv_sorted:
                    if any(True for route in changed_routes if route in swap[2]):
                        del swap_improv[swap[1]]
            
            # If the best improvement is a swap improvement, compute the best swap
            if best_swap_improv >= best_move_improv:
                self.swap_nodes(*swap_improv_sorted[0][1])
                operations+=1 # increase number of operations by 1
                print('Swapped {} with {}'.format(swap_improv_sorted[0][1][0].data.name,
                                                  swap_improv_sorted[0][1][1].data.name))
                
                # Keep track of the routes that have changed during the swap operation
                changed_routes = list(swap_improv_sorted[0][2])
                
                # Delete moves in changed_routes from move_improv dictionary, as they have to be computed again in 
                # next iteration
                for move in move_improv_sorted:
                    if any(True for route in changed_routes if route in move[2]):
                        del move_improv[move[1]]
                
                # Delete swaps in changed_routes from swap_improv dictionary, as they have to be computed again in 
                # next iteration
                for swap in swap_improv_sorted:
                    if any(True for route in changed_routes if route in swap[2]):
                        del swap_improv[swap[1]]
                        
        print('\nOptimized distance = {} in {} move/swap operations'.format(solver.total_distance, operations))
        
        # Make dictionary with route number and key and route instance as value
        routes = {}
        i = 1
        for route in self.planned_routes:
            routes[i] = route
            i += 1
        
        return routes

In [15]:
# Instantiate solver instance for local search improvement heuristic
solver = Solver(planned_routes_dll)
planned_routes_2 = solver.optimize() # Solve local search improvement heuristic 

# Save results to Excel file
df_q21 = to_Excel(planned_routes_2, 2, file_name='../excel-files/Ex2.2-1273195.xls')

Moved EMTE ST ANTHONIS after EMTE Montfort FR
Swapped EMTE GROENLO with EMTE RIJSSEN VEENESLAGEN
Swapped EMTE RIJSSEN VEENESLAGEN with EMTE BATHMEN FR
Swapped EMTE UDENHOUT with EMTE KAATSHEUVEL
Moved EMTE Eindhoven victoriapark after EMTE VESSEM FR
Moved EMTE GROENLO after EMTE HEADQUARTERS VEGHEL
Moved EMTE KRUININGEN FR after EMTE HOEK FR
Moved EMTE S GRAVENPOLDER FR after EMTE HOEK FR
Moved EMTE LOON OP ZAND after EMTE HEADQUARTERS VEGHEL
Moved EMTE GROOT AMMERS after EMTE HOORNAAR FR
Moved EMTE TILBURG BESTERDRING after EMTE GILZE
Swapped EMTE ENTER FR with EMTE RIJSSEN OPSTALLSTR
Moved EMTE Dongen after EMTE UDENHOUT
Moved EMTE TILBURG WAGNERPLEIN after EMTE UDENHOUT
Swapped EMTE OSSENDRECHT FR with EMTE PUTTE (NB)
Moved EMTE SCHAIJK after EMTE WIJCHEN
Swapped EMTE VLISSINGEN FR with EMTE KOUDEKERKE FR
Swapped EMTE CUIJK with EMTE LOBITH
Swapped EMTE ST MICHIELSGESTEL with EMTE DEN BOSCH
Moved EMTE OUDENBOSCH after EMTE KRUININGEN FR
Moved EMTE OOLTGENSPLAAT FR after EMTE KRUININ

## Exercise 2.3

In [16]:
def calc_dist(loc1, loc2):
    """Defines rounded distance (km) between two locations"""
    dist = round(haversine(loc1.coords, loc2.coords))
    return dist

In [17]:
DIST_MATRIX = np.zeros((len(locations), len(locations)))

for location_1 in locations:
    for location_2 in locations:
        DIST_MATRIX[location_1.nr, location_2.nr] = calc_dist(location_1, location_2)

In [18]:
STORES['is_jumbo'] = STORES.apply(lambda row: row['Type']=='Jumbo', axis=1) 
STORES.head()

,Name,Address,Postal code,City,Lat,Long,Type,is_jumbo
City Nr.,,,,,,,,
0,EMTE HEADQUARTERS VEGHEL,CORRIDOR 11,5466RB,VEGHEL,51.606702,5.528046,NaN,False
1,EMTE ARKEL,DR H DE VRIESPLN 14,4241BW,ARKEL,51.864000,4.993040,Coop,False
2,EMTE ARNEMUIDEN FR,CLASINASTR 5,4341ER,ARNEMUIDEN,51.500010,3.677280,Jumbo,True
3,EMTE BATHMEN FR,LARENSEWG 18,7437BM,BATHMEN,52.249060,6.289990,Jumbo,True
4,EMTE BEEK EN DONK,HEUVELPLN 73,5741JJ,BEEK EN DONK,51.529300,5.632300,Jumbo,True


In [29]:
toolbox = base.Toolbox()
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.10)

In [30]:
solution_ex_2 = []

for route in planned_routes_2.values():
    temp = route.start.next
    while temp != route.end:
        solution_ex_2.append(temp.data.nr)
        temp = temp.next

solution_ex_2 = [loc-1 for loc in solution_ex_2]
        
init_pop = [deepcopy(solution_ex_2) for i in range(300)]

mut_prob = 0.95
shuffle_prob = 0.50

[toolbox.mutate(ind) if random.random() < mut_prob else ind for ind in init_pop];
[random.shuffle(ind) if random.random() < shuffle_prob else ind for ind in init_pop];

In [32]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def initIndividual(icls, content):
    return icls(content)

def initPopulation(pcls, ind_init, init_pop):
    return pcls(ind_init(ind) for ind in init_pop)

toolbox = base.Toolbox()

toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, init_pop)

In [33]:
# HELPER FUNCTIONS FOR EVALUATE FUNCTION

HQ = locations[0].nr
MAX_WORKING_MINS = 11*60
MAX_OPENING_MINS = 8*60
MAX_OPENING_MINS = 8*60
SPEED_KMH = 90

def total_distance(route, full_route=False):
    route_to_check = deepcopy(route)
    
    if full_route:
        route_to_check.insert(0, HQ)
        route_to_check.append(HQ)
    
    distances = [DIST_MATRIX[route_to_check[idx], route_to_check[idx+1]] 
                 for idx, _ in enumerate(route_to_check[:-1])]
    
    return distances
    

def travel_time(route, full_route=False):
    """
    Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route
    """
    
    def dist_to_min(km):
        """Defines the duration of a route in minutes"""
        speed_kmm = (SPEED_KMH)/60
        minutes = round(km/speed_kmm)
        return minutes

    minutes_between_locs = [dist_to_min(dist) for dist in total_distance(route, full_route=full_route)]
    travel_time = sum(minutes_between_locs)
    return travel_time

def visiting_time(route):
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    time = 30 if is_jumbo else 20
    return len(route)*time

def working_hours_constraint(route):
    """Defines if route meets the constraint of John's working hours"""
    return (travel_time(route, full_route=True) + visiting_time(route)) <= MAX_WORKING_MINS

def opening_hours_constraint(route):
    """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
    return (travel_time(route) + visiting_time(route)) <= MAX_OPENING_MINS

def store_type_constraint(route):
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    
    for location in route[1:]:
        if STORES.loc[location, 'is_jumbo'] != is_jumbo:
            return False
    
    return True

def is_valid(route):
    return working_hours_constraint(route) and opening_hours_constraint(route) and store_type_constraint(route)

In [34]:
def evaluate_schedule(individual):
    schedule = []
    route = []
    for loc in individual:
        route.append(loc+1)
        if not is_valid(route):
            route.remove(loc+1)
            schedule.append(deepcopy(route))
            route = [loc+1]
    schedule.append(deepcopy(route))
    
    distances = [sum(total_distance(route, full_route=True)) for route in schedule]
    
    return sum(distances),

In [35]:
toolbox.register("evaluate", evaluate_schedule)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [36]:
def find_second_smallest(my_list):
    return sorted(list(set(my_list)))[1]

def find_smallest(my_list):
    return sorted(list(my_list))[0]

def mean_dist(my_list):
    return np.mean([dist for dist, nr_routes in my_list])

def std_dist(my_list):
    return np.std([dist for dist, nr_routes in my_list])

def find_largest(my_list):
    return sorted(list(my_list))[-1]

In [37]:
def main():
    pop = toolbox.population_guess()
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("2nd smallest", find_second_smallest)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=10000, 
                                   stats=stats, halloffame=hof, verbose=True)

In [ ]:
main()

gen	nevals	avg    	std    	min 	max  	2nd smallest
0  	300   	10866.3	3998.15	2838	16090	(4507.0,)   
1  	230   	8089.89	2966.17	2838	15700	(2924.0,)   
2  	224   	6357.49	1490.3 	2838	13405	(2924.0,)   
3  	217   	5730.93	1406.5 	2838	10180	(2924.0,)   
4  	233   	5007.56	1237.57	2838	8966 	(2866.0,)   
5  	226   	4455.68	1151.98	2838	9657 	(2866.0,)   
6  	227   	4066.64	1215.2 	2838	8834 	(2866.0,)   
7  	238   	3656.99	1006.89	2838	7322 	(2866.0,)   
8  	216   	3569.01	1107.93	2838	7750 	(2866.0,)   
9  	236   	3457.56	1029.79	2838	8059 	(2863.0,)   
10 	236   	3395.87	1074.83	2838	7896 	(2866.0,)   
11 	221   	3295.76	977.425	2838	7194 	(2866.0,)   
12 	243   	3349.98	1099.4 	2838	7025 	(2866.0,)   
13 	234   	3301.63	1045.4 	2838	7686 	(3228.0,)   
14 	238   	3346.33	1037.91	2838	6867 	(3055.0,)   
15 	241   	3347.5 	1098.01	2838	7856 	(3294.0,)   
16 	224   	3370.28	1104.89	2838	7753 	(2865.0,)   
17 	211   	3360.09	1088.14	2838	7477 	(2865.0,)   
18 	236   	3346.66	1051.37	2838

## TRYOUT : DIFFERENT GA FOR JUMBO AND COOP

In [ ]:
def calc_dist(loc1, loc2):
    """Defines rounded distance (km) between two locations"""
    dist = round(haversine(loc1.coords, loc2.coords))
    return dist

In [ ]:
DIST_MATRIX = np.zeros((len(locations), len(locations)))

for location_1 in locations:
    for location_2 in locations:
        DIST_MATRIX[location_1.nr, location_2.nr] = calc_dist(location_1, location_2)

In [339]:
STORES['is_jumbo'] = STORES.apply(lambda row: row['Type']=='Jumbo', axis=1) 
STORES.head()

,Name,Address,Postal code,City,Lat,Long,Type,is_jumbo,GA_nr
City Nr.,,,,,,,,,
0,EMTE HEADQUARTERS VEGHEL,CORRIDOR 11,5466RB,VEGHEL,51.606702,5.528046,NaN,False,None
1,EMTE ARKEL,DR H DE VRIESPLN 14,4241BW,ARKEL,51.864000,4.993040,Coop,False,30
2,EMTE ARNEMUIDEN FR,CLASINASTR 5,4341ER,ARNEMUIDEN,51.500010,3.677280,Jumbo,True,2
3,EMTE BATHMEN FR,LARENSEWG 18,7437BM,BATHMEN,52.249060,6.289990,Jumbo,True,31
4,EMTE BEEK EN DONK,HEUVELPLN 73,5741JJ,BEEK EN DONK,51.529300,5.632300,Jumbo,True,71


In [340]:
solution_ex_2 = {'Jumbo': [],
                 'Coop': []}

for route in planned_routes_2.values():
    temp = route.start.next
    is_jumbo = temp.data.is_jumbo
    while temp != route.end:
        if is_jumbo:
            solution_ex_2['Jumbo'].append(temp.data.nr)
        else:
            solution_ex_2['Coop'].append(temp.data.nr)
        temp = temp.next

# solution_ex_1 = {'Jumbo': [], 
#                  'Coop': []}
        
# for route in planned_routes_1.values():
#     is_jumbo = route.inner_route[0].is_jumbo
#     for loc in route.inner_route:
#         if is_jumbo:
#             solution_ex_1['Jumbo'].append(loc.nr)
#         else:
#             solution_ex_1['Coop'].append(loc.nr)
            
            
# solution_ex_2 = [loc-1 for loc in solution_ex_2]
        
# init_pop = [deepcopy(solution_ex_2) for i in range(300)]
# shuffle_chance = 0.95

# [random.shuffle(ind) if random.random() < shuffle_chance else ind for ind in init_pop];

In [341]:
# jumbos_ex_1 = solution_ex_1['Jumbo']
# coops_ex_1 = solution_ex_1['Coop']

jumbos_ex_2 = solution_ex_2['Jumbo']
coops_ex_2 = solution_ex_2['Coop']

In [342]:
# jumbos_ex_1 = list(zip(jumbos_ex_1, [nr for nr in range(len(jumbos_ex_1))]))
# coops_ex_1 = list(zip(coops_ex_1, [nr for nr in range(len(coops_ex_1))]))

jumbos_ex_2 = list(zip(jumbos_ex_2, [nr for nr in range(len(jumbos_ex_2))]))
coops_ex_2 = list(zip(coops_ex_2, [nr for nr in range(len(coops_ex_2))]))

In [343]:
# JUMBO_GA_NR_1 = {ga_nr: nr for nr, ga_nr in jumbos_ex_1}
# COOP_GA_NR_1 = {ga_nr: nr for nr, ga_nr in coops_ex_1}

JUMBO_GA_NR = {ga_nr: nr for nr, ga_nr in jumbos_ex_2}
COOP_GA_NR = {ga_nr: nr for nr, ga_nr in coops_ex_2}

In [344]:
# jumbos_ex_1 = [ga_nr for nr, ga_nr in jumbos_ex_1]
# coops_ex_1 = [ga_nr for nr, ga_nr in coops_ex_1]

jumbos_ex_2 = [ga_nr for nr, ga_nr in jumbos_ex_2]
coops_ex_2 = [ga_nr for nr, ga_nr in coops_ex_2]

In [345]:
# HELPER FUNCTIONS FOR EVALUATE FUNCTION

HQ = locations[0].nr
MAX_WORKING_MINS = 11*60
MAX_OPENING_MINS = 8*60
MAX_OPENING_MINS = 8*60
SPEED_KMH = 90

def total_distance(route, full_route=False):
    route_to_check = deepcopy(route)
    
    if full_route:
        route_to_check.insert(0, HQ)
        route_to_check.append(HQ)
    
    distances = [DIST_MATRIX[route_to_check[idx], route_to_check[idx+1]] 
                 for idx, _ in enumerate(route_to_check[:-1])]
    
    return distances
    

def travel_time(route, full_route=False):
    """
    Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route
    """
    
    def dist_to_min(km):
        """Defines the duration of a route in minutes"""
        speed_kmm = (SPEED_KMH)/60
        minutes = round(km/speed_kmm)
        return minutes

    minutes_between_locs = [dist_to_min(dist) for dist in total_distance(route, full_route=full_route)]
    travel_time = sum(minutes_between_locs)
    return travel_time

def visiting_time(route):
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    time = 30 if is_jumbo else 20
    return len(route)*time

def working_hours_constraint(route):
    """Defines if route meets the constraint of John's working hours"""
    return (travel_time(route, full_route=True) + visiting_time(route)) <= MAX_WORKING_MINS

def opening_hours_constraint(route):
    """Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route"""
    return (travel_time(route) + visiting_time(route)) <= MAX_OPENING_MINS

def is_valid(route):
    return working_hours_constraint(route) and opening_hours_constraint(route)

In [346]:
def evaluate_schedule_jumbo(individual):
    schedule = []
    route = []
    for loc in individual:
        loc = JUMBO_GA_NR[loc]
        route.append(loc)
        if not is_valid(route):
            route.remove(loc)
            schedule.append(deepcopy(route))
            route = [loc]
    schedule.append(deepcopy(route))
    
    distances = [sum(total_distance(route, full_route=True)) for route in schedule]

    return sum(distances),

In [347]:
def evaluate_schedule_coop(individual):
    schedule = []
    route = []
    for loc in individual:
        loc = COOP_GA_NR[loc]
        route.append(loc)
        if not is_valid(route):
            route.remove(loc)
            schedule.append(deepcopy(route))
            route = [loc]
    schedule.append(deepcopy(route))
    
    distances = [sum(total_distance(route, full_route=True)) for route in schedule]

    return sum(distances),

In [348]:
evaluate_schedule_jumbo(jumbos_ex_1)

(1552.0,)

In [349]:
evaluate_schedule_coop(coops_ex_2)

(1286.0,)

### GA Jumbo

In [350]:
toolbox = base.Toolbox()
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.10)

In [351]:
init_pop_jumbo = [deepcopy(jumbos_ex_2) for i in range(300)]

mut_prob = 0.95
shuf_prob = 0.50
[toolbox.mutate(ind) if random.random() < mut_prob else ind for ind in init_pop_jumbo];
[random.shuffle(ind) if random.random() < shuf_prob else ind for ind in init_pop_jumbo];

In [352]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def initIndividual(icls, content):
    return icls(content)

def initPopulation(pcls, ind_init, init_pop):
    return pcls(ind_init(ind) for ind in init_pop)

toolbox = base.Toolbox()

toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, init_pop_jumbo)

In [353]:
toolbox.register("evaluate", evaluate_schedule_jumbo)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=5)

In [354]:
def find_second_smallest(my_list):
    return sorted(list(set(my_list)))[1]

In [355]:
def main():
    pop = toolbox.population_guess()
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("2nd smallest", find_second_smallest)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=10000, 
                                   stats=stats, halloffame=hof, verbose=True)

In [356]:
main()

gen	nevals	avg    	std   	min 	max 	2nd smallest
0  	300   	5768.66	2009.5	1552	8416	(2559.0,)   
1  	215   	3780.58	1145.18	1552	7997	(1557.0,)   
2  	226   	2860.33	813.909	1552	5147	(1557.0,)   
3  	228   	2206.44	676.053	1552	4947	(1557.0,)   
4  	234   	2009.91	770.062	1552	5073	(1557.0,)   
5  	237   	1795.51	626.762	1552	5390	(1557.0,)   
6  	220   	1804.75	607.181	1552	4170	(1678.0,)   
7  	225   	1828.8 	615.045	1552	5081	(1777.0,)   
8  	228   	1886.37	711.533	1552	4814	(1774.0,)   
9  	209   	1836.1 	638.231	1552	4402	(2001.0,)   
10 	241   	1838.77	641.247	1552	4323	(1567.0,)   
11 	228   	1840.93	652.8  	1552	4432	(2165.0,)   
12 	228   	1808.64	638.025	1552	4839	(1566.0,)   
13 	237   	1834.78	624.378	1552	4205	(1606.0,)   
14 	217   	1809.29	597.709	1552	4407	(1767.0,)   
15 	217   	1741.03	517.133	1552	4516	(1577.0,)   
16 	222   	1834.77	665.996	1552	4356	(1595.0,)   
17 	228   	1861.49	677.228	1552	5353	(1602.0,)   
18 	213   	1890.41	718.585	1552	4616	(1578.0,)   
19